# IMPORTATION DES BIBLIOTHEQUES

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import SnowballStemmer

# Télécharger les ressources nécessaires de NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# DEFINITION DES DICTIONNAIRE NECESSAIRE

In [ ]:
error_medical={
    "Ttt": "Traitement",
    'Pat': 'Patient',
}
terme_medicals = {
    "corticoïdes" : "glucocorticoides",
    "azithromycine" : "azithromycine",
}
expressions_composes = {
    "++fébrile" : "hyperthermique",
}
abreviations = {
    "IV" : "intraveineux",
    "COVID-19" : "coronavirus",

}


# NETTOYAGE DU TEXT

In [ ]:



def nettoyer_texte(texte):


    for items,abreviation in abreviations.items():
        texte = texte.replace(items, abreviation)
    for items,terme_medical in terme_medicals.items():
        texte = texte.replace(items, terme_medical)
    for items,expressions_compose in expressions_composes.items():
        texte = texte.replace(items, expressions_compose)
    

    # Supprimer les dates (format JJ/MM/AA ou JJ/MM/AAAA)
    texte = re.sub(r'\d{2}/\d{2}/\d{2,4}', '', texte)

    # remove punctuation except for . between numbers
    texte = re.sub(r'(?<!\d)\.(?!\d)', '', texte)
    
    # Supprimer les valeurs numériques avec unités (ex: "125mg/L", "500mg/J"), mais conserver les nombres sans unités
    texte = re.sub(r'\d+(\.\d+)?[a-zA-Z/]+', '', texte)
    
    # Supprimer les caractères spéciaux (ex: "#", "°",'º',etc.)
    texte = re.sub(r'[^a-zA-Z0-9\s.,-]', '', texte)    

    for error, signification in error_medical.items():
        texte = texte.replace(error, signification)
    
    return texte.strip()

# Exemple
texte_medical = """
Pat. 45ans admis le 12/03/25 pour COVID-19 sévère.
CRP: 125mg/L (N<5) - Fièvre à 39.2°C.
Ttt par corticoïdes IV + azithromycine 500mg/J. 500mg
#Attention: allergie à la pénicilline!
"""
texte_nettoye = nettoyer_texte(texte_medical)
print(texte_nettoye)


Patient  admis le  pour coronavirus svre
CRP  N5 - Fivre  39.2C
Traitement par glucocorticoides intraveineux  azithromycine  
Attention allergie  la pnicilline


# CONSERVCATION DES DOSES ET DES MOTS COMPOSÉ

In [88]:
def conserve_important_words(texte):
    # Conserver les concepts composés
    texte = re.sub(r'(\w+-\w+)', r' \1 ', texte)
    
    # Séparer les doses
    texte_test = re.sub(r'(\d+)(mg|g|ml|L|J|j|l)(/|\s)?(mg|min|g|ml|L|J|j|l)?', r"[['\1','\2','\3','\4']]", texte)
    texte_test1 = re.sub(r'(\d+)(mg|g|ml|L|J|j|l)', r"[['\1','\2']]", texte)
    
    # get the dosez with uniut/unit and unite
    doses = re.findall(r'\[\[\'(\d+)\',\'([a-zA-Z]+)\',\'([/ ]?)\',\'([a-zA-Z]+)\'\]\]', texte_test)

    # get doses with unite (mg|g|ml|L|J|j|l)
    doses =doses + re.findall(r'\[\[\'(\d+)\',\'([a-zA-Z]+)\'\]\]', texte_test1)


    # get concepts composés
    concepts_composes = re.findall(r'(\w+-\w+)', texte)

    # get the doses and concepts_composes in one list
    test = []
    for dose in doses:
        test.append(dose)
    for concept in concepts_composes:
        test.append(concept)

    return test

# Exemple
words_conserved = conserve_important_words(texte_medical)
print(words_conserved)

[('125', 'mg', '/', 'L'), ('500', 'mg', '/', 'J'), ('125', 'mg'), ('500', 'mg'), ('500', 'mg'), 'COVID-19']


# TOKENISATION DU TEXT

In [ ]:
def supprimer_stopwords(mots):
    stop_words = set(stopwords.words('french'))
    # words = word_tokenize(text)
    filtered_text = [word for word in mots if word not in stop_words]
    return filtered_text
def tokeniser_texte(texte):
    # Tokenisation des phrases
    phrases = sent_tokenize(texte, language='french')
    
    # Tokenisation des mots
    tokens = [word_tokenize(phrase, language='french') for phrase in phrases]
    
    # Aplatir la liste de listes
    tokens = [item for sublist in tokens for item in sublist]
    
    # Supprimer les stopwords
    tokens = supprimer_stopwords(tokens)

    
    return tokens

tokenized_text = tokeniser_texte(texte_nettoye)
print(tokenized_text)

Tokens avant suppression des stopwords: ['Patient  admis le  pour coronavirus svre\nCRP  N5 - Fivre  39.2C\nTraitement par glucocorticoides intraveineux  azithromycine  \nAttention allergie  la pnicilline']
['Patient', 'admis', 'coronavirus', 'svre', 'CRP', 'N5', '-', 'Fivre', '39.2C', 'Traitement', 'glucocorticoides', 'intraveineux', 'azithromycine', 'Attention', 'allergie', 'pnicilline']


# NORMALISATION DU TEXT

In [ ]:
def normaliser_texte(words,words_conserved):
    stemmer = SnowballStemmer("french")
    
    # Flatten the list to handle tuples and strings
    flattened_words = []
    for word in words:
        if isinstance(word, tuple):
            flattened_words.extend(word)  # Add elements of the tuple
        else:
            flattened_words.append(word)  # Add the string directly

    # Apply stemming only to strings
    word_normalized = [stemmer.stem(word.lower()) for word in flattened_words if isinstance(word, str)]
    
    # Normaliser les termes médicaux
    for terme, signification in terme_medicals.items():
        word_normalized = [signification if word == terme.lower() else word for word in word_normalized]

    # Normaliser les expressions composées
    for expression, signification in expressions_composes.items():
        word_normalized = [signification if word == expression.lower() else word for word in word_normalized]

    # Normaliser les abréviations
    for abrev, signification in abreviations.items():
        word_normalized = [signification if word == abrev.lower() else word for word in word_normalized]

    word_normalized += words_conserved
    
    return word_normalized

# Exemple
print(normaliser_texte(tokenized_text, words_conserved))


['patient', 'admis', 'coronavirus', 'svre', 'crp', 'n5', '-', 'fivr', '39.2c', 'trait', 'glucocorticoid', 'intravein', 'azithromycin', 'attent', 'allerg', 'pnicillin', ('125', 'mg', '/', 'L'), ('500', 'mg', '/', 'J'), ('125', 'mg'), ('500', 'mg'), ('500', 'mg'), 'COVID-19']


# NEW EXEMPLES

In [ ]:
data = {
    "texte1" : """Pat. 60ans hospitalisé le 15/04/2023 pour pneumopathie.
        CRP à 240mg/L. Ttt initial par amoxicilline 1g/J.
        Fièvre persistante à 38.5°C. Scanner réalisé.
        """,
    "texte2" : """Pat. 33ans. Chute le 01/01/25. Scanner cérébral normal.
        Température à 37.8ºC. Ttt: paracétamol 500mg.
        """,
    "texte3" : """Admission d’un homme de 72ans le 02/02/22.
        Douleurs thoraciques. ECG normal. Troponine: 0.04ng/mL.
        Ttt: surveillance + aspirine 75mg/j.
        """,
    "texte4" : """Pat. 25ans, suivi pour asthme. Crise aiguë le 04/03/2025.
        Oxygène à 3L/min. Corticoïdes 100mg IV administrés.
        """,
    "texte5" : """Pat. 50ans diabétique. Consultation du 10/10/2020.
        HbA1c: 9.2%. Ttt modifié: metformine 1000mg/j.
        Régime conseillé. Poids: 95kg.
        """
}

## NETTOYER -> CONSERVER LES DOSES ET MOTS COMPOSÉ -> TOKENISER LE TEXT -> NORMALISER LE TEXT -> VISUALISER LES RESULTAT

In [ ]:
# Exemple d'utilisation
for key, value in data.items():
    print(f"{key}: {value}")
    texte_nettoyes = nettoyer_texte(value)
    print("Texte nettoyé:", texte_nettoyes)
    conserved_words = conserve_important_words(value)
    print("Dose et Mots composé :", conserved_words)
    tokenized_text = tokeniser_texte(texte_nettoye)
    print("Texte tokenisé:", tokenized_text)
    texte_normalise = normaliser_texte(tokenized_text,conserved_words)
    print("Text Normaliser :", texte_normalise)
    print("\n")

texte1: Pat. 60ans hospitalisé le 15/04/2023 pour pneumopathie.
        CRP à 240mg/L. Ttt initial par amoxicilline 1g/J.
        Fièvre persistante à 38.5°C. Scanner réalisé.
        
Texte nettoyé: Patient  hospitalis le  pour pneumopathie
        CRP   Traitement initial par amoxicilline 
        Fivre persistante  38.5C Scanner ralis
Dose et Mots composé : [('240', 'mg', '/', 'L'), ('1', 'g', '/', 'J'), ('240', 'mg'), ('1', 'g')]
Tokens avant suppression des stopwords: ['Patient  admis le  pour coronavirus svre\nCRP  N5 - Fivre  39.2C\nTraitement par glucocorticoides intraveineux  azithromycine  \nAttention allergie  la pnicilline']
Texte tokenisé: ['Patient', 'admis', 'coronavirus', 'svre', 'CRP', 'N5', '-', 'Fivre', '39.2C', 'Traitement', 'glucocorticoides', 'intraveineux', 'azithromycine', 'Attention', 'allergie', 'pnicilline']
Text Normaliser : ['patient', 'admis', 'coronavirus', 'svre', 'crp', 'n5', '-', 'fivr', '39.2c', 'trait', 'glucocorticoid', 'intravein', 'azithromycin', 